In [0]:
!pip install hazm
import pandas as pd 
from hazm import * 

     |████████████████████████████████| 317kB 4.8MB/s 
     |████████████████████████████████| 1.4MB 6.2MB/s 
     |████████████████████████████████| 235kB 24.1MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394471 sha256=f03af7db5ec1109b3db0407fc751a0dbaa9d2b766d4224a4b248dfe1245beaaf
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=153524 sha256=66d8fb74bb29696ca94298c36cdd683ff896b959f3c3a5af23fd882dc9e44c45
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


<html>
  <h3>
    I've run SQL files in my LopTop, convert tables to .csv and uploaded them to my google drive. At cell bellow i've read them from drive .
  </h3>
</html>

In [0]:
social = pd.read_csv('/content/drive/My Drive/parto tech/social.csv')
economy = pd.read_csv('/content/drive/My Drive/parto tech/economy.csv')
politics = pd.read_csv('/content/drive/My Drive/parto tech/politics.csv')

In [0]:
al = economy.append(social,ignore_index=True)

In [0]:
al = al.append(politics,ignore_index=True)

In [0]:
def remove_backslashes(mystring) :
  ret =  mystring.replace('\\n',' ')
  ret =  ret.replace('\\r\\n',' ')
  ret =  ret.replace('\\n\\r',' ')
  ret =  ret.replace('\n',' ')
  ret =  ret.replace('\r\n',' ')
  ret =  ret.replace('\n\r',' ')
  ret =  ret.replace('\r',' ')
  return ret.replace('\\r',' ')


In [0]:
al.loc[:]['train_text'] = al['train_text'].apply(remove_backslashes)

In [0]:
al.to_pickle('/content/drive/My Drive/parto tech/all_together.pkl')

<html>
  <h3>
    Till this step i read csv files, append them together then apply <i><b>remove_backslashes</b></i> function to <i>train_text</i> column
  </h3>
</html>

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report ,f1_score

<html>
  <h3>Cell below contains three steps :</h3>
  <ol>
    <li>Defining a function for preprocessing the text. First i extract sentences then iterate for each sentence and normalize, lemmatize and stem all words in a senctence. </li>
    <li>Creating pipeline of Count vectorizer, Tfidf transformer and model</li>
    <li>Fitting the model</li>
  </ol>
</html>

In [0]:
def preprocessing(text) :
  stemmer = Stemmer()
  lemmatizer = Lemmatizer()
  normalizer = Normalizer()

  stop_words = stopwords_list()
  all_sentences = sent_tokenize(text)
  new_sents = []
  for sent in all_sentences :
    all_words = word_tokenize(normalizer.normalize(sent))
    new_sent = [lemmatizer.lemmatize(stemmer.stem(word)).split('#')[0] for word in all_words if word not in stop_words]
    new_sent = ' '.join(new_sent) 
    new_sents.append(new_sent)
  
  return ' '.join(new_sents)


pipline = Pipeline([
        ('bag of words', CountVectorizer(preprocessor=preprocessing)),
        ('tfidf', TfidfTransformer()),
        ('model', KNeighborsClassifier(n_neighbors=2))
    ])

msg_train, msg_test, label_train, label_test = train_test_split(al['train_text'], al['cat_id'], test_size=0.3,
                                                                    random_state=101)

pipline.fit(msg_train, label_train)
print(pipline.score(msg_test, label_test))

0.9854014598540146


In [0]:
y_pred = pipline.predict(msg_test)

In [0]:
print(classification_report(y_true=label_test,y_pred=y_pred))

              precision    recall  f1-score   support

           3       1.00      0.94      0.97        18
           5       0.98      1.00      0.99        52
           7       0.99      0.99      0.99        67

    accuracy                           0.99       137
   macro avg       0.99      0.98      0.98       137
weighted avg       0.99      0.99      0.99       137



In [0]:
import pickle 

file_name = '/content/drive/My Drive/parto tech/model_hazm.pkl'
pickle.dump(pipline,open(file_name,'wb'))